In [1]:
# see https://towardsdatascience.com/how-do-i-extract-nested-data-in-python-4e7bed37566a

In [2]:
import pandas as pd
import requests
import json
from config import api_key
from pandas import json_normalize
import os

<ipython-input-2-dfbaaafc61f7>:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
# pretty print the JSON
from pprint import pprint

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 100)
# use below code to see full text of the columns; by providing None as the 2nd parameter, 
# we are setting the max characters allowed before truncation to unlimited
pd.set_option("display.max_colwidth", None)

In [4]:
os.getcwd()


'C:\\Users\\Jenkir\\Desktop\\cherry-tree-lane-analytics\\INDUSTRIES\\Materials-GICsector-EDA'

In [5]:
os.chdir('C:\\Users\\Jenkir\\Desktop\\cherry-tree-lane-analytics\\INDUSTRIES\\Materials-GICsector-EDA')
os.getcwd()

'C:\\Users\\Jenkir\\Desktop\\cherry-tree-lane-analytics\\INDUSTRIES\\Materials-GICsector-EDA'

In [6]:
# see https://www.youtube.com/watch?v=r7t71t5993s  for how to add quotes and commas to values in Excel

headers = {
    "Api-key": api_key
          }

ids = ["193281",
"192130",
"244885",
"193361",
"191905",
"197846",
"193607",
"242957",
"191623",
"192198",
"192155",
"191658",
"208713",
"192085",
"200039",
"193529",
"192433",
"190075",
"204280",
"206823",
"204783",
"194815",
"240133"
      ]

api_data = []

for i in ids:
    details = requests.get("https://api.esganalytics.io/api/company/details/?company_id={}".format(i), headers=headers)
    
    if details.status_code == 200:
        api_data.append(details.json())
    else:
        print("Request to { failed}".format(i))
   
pprint(api_data)

KeyError: ' failed'

In [ ]:
json_data = json.dumps(api_data)
dict_data = json.loads(json_data)
pprint(dict_data)

In [ ]:
print(type(dict_data))


In [ ]:
print(type(api_data))

In [ ]:
print(len(dict_data))

In [ ]:
print(len(api_data))

In [ ]:
#pip install flatten_json

In [ ]:
# see https://github.com/amirziai/flatten
#from flatten_json import flatten

In [ ]:
#df_flatten = pd.DataFrame([flatten(dict_data)['id']])
#df_flatten

In [ ]:
print(dict_data[0])

In [ ]:
# see https://stackoverflow.com/questions/47242845/pandas-json-normalize-with-very-nested-json


df_detail2 = pd.json_normalize(dict_data, 'company_detail')
df_detail2

In [ ]:
df_detail2.columns

In [ ]:
# drop unnecessary columns
df_detail = df_detail2.drop(columns=['description', 'url', 'pos_neg_ratio', 'esg_beta_tseries'])
df_detail

In [ ]:
# sort by company name & will use later when joining with next df bc there are no common columns; use ignore_index=True to reset index & keep new index while discarding old one
df_detail_F = df_detail.sort_values(by=["company_name"],ignore_index=True)
df_detail_F 

In [ ]:
df_detail_F.info()

In [ ]:
df_securities = pd.json_normalize(dict_data, 'securities')
df_securities

In [ ]:
df_securities_US = df_securities.loc[df_securities['currency_code'] == "USD"]
df_securities_US

In [ ]:
# select any duplicate rows 
dupes = df_securities_US[df_securities_US.duplicated("code")]
dupes


In [ ]:
# drop duplicate rows
securities_final2= df_securities_US.drop_duplicates("code")
securities_final2

In [ ]:
securities_final = securities_final2.sort_values(by=["code"], ignore_index=True)
securities_final

In [ ]:
# merge the df that has company details with the df that has securities info; see https://www.statology.org/pandas-merge-on-index/

df_merged = pd.merge(df_detail_F, securities_final, left_index=True, right_index=True)
df_merged

In [ ]:
df_merged[["street_address", "City", "State", "Country", "zip"]] = df_merged["address"].str.split(",", expand=True)
df_merged

In [ ]:
df_merged.columns

In [ ]:
df_merged_temp = df_merged.rename(columns={'code': "symbol",
                                           'id': 'unique_id'
                                        })
df_merged_temp

In [ ]:
df_merged_temp.columns

In [ ]:
# reorder the columns so that they match other datasets
df_details_secur = df_merged_temp[['unique_id', 'company_name', 'gicSector', 'gicSubIndustry','company_pulse', 
        'esg_beta', 'City', 'State', 'Country','zip','exchange', 'symbol', 'isin',
       'cusip', 'securities_type', 'currency_code']]

df_details_secur

In [ ]:
df_details_secur.dtypes

In [ ]:
convert = {"unique_id": str
          }
df_to_csv = df_details_secur.astype(convert)
df_to_csv.info()

In [ ]:
# Check to see how many unique company names there are in our results
df_to_csv.company_name.nunique(dropna=True)

In [ ]:
# Export file as a CSV, without the Pandas index, but with the header
#df_to_csv.to_csv("test.csv", index = False, header=True)